In [90]:
from fractions import Fraction
from itertools import combinations, combinations_with_replacement, product, permutations
# from typing import *
import math
# import random
from collections import Counter, defaultdict

In [ ]:
Die = {1,2,3,4,5,6}
Dtype = {'FLAME', 'BLAZE', 'FIERY_SOUL', 'METEOR'}
DDict = {1: 'FLAME', 2: 'FLAME', 3: 'FLAME', 4: 'BLAZE', 5: 'FIERY_SOUL', 6: 'METEOR'}

fireball1 = ['FLAME', 'FLAME', 'FLAME']
fireball2 = ['FLAME', 'FLAME', 'FLAME', 'FLAME']
fireball3 = ['FLAME', 'FLAME', 'FLAME', 'FLAME', 'FLAME']
pyroblast = ['FLAME', 'FLAME', 'FLAME', 'METEOR']
burning_soul = ['FIERY_SOUL', 'FIERY_SOUL']
combustion = ['FLAME', 'BLAZE', 'FIERY_SOUL', 'METEOR']
meteorite = ['METEOR', 'METEOR', 'METEOR', 'METEOR']
ultimate = ['METEOR', 'METEOR', 'METEOR', 'METEOR', 'METEOR']

hot_streak = [[1,2,3,4], [2,3,4,5], [3,4,5,6]]
ignite = [[1,2,3,4,5], [2,3,4,5,6]]

attack_list = {
    "Fireball1" : fireball1,
    "Fireball2" : fireball2,
    "Fireball3" : fireball3,
    "Pyroblast" : pyroblast,
    "Burning Soul" : burning_soul,
    "Combustion" : combustion,
    "Meteorite" : meteorite,
    "Ultimate" : ultimate,
    }

straight_list = {"Hot Streak" : hot_streak, "Ignite" : ignite}


In [ ]:
# Takes the dice as numbers, and returns a list of strings of the types
def dieToType(die:list[int]) -> list[str]:
    return [DDict[d] for d in die]

# Given the dice as numbers, returns the valid attacks
def valid_attacks(die:list[int]) -> list[str]:
    dtypes = dieToType(die)
    valid_attacks = [
        att for att, att_list in attack_list.items() if all(dtypes.count(x) >= att_list.count(x) for x in set(att_list))
    ]
    straights = [
        att for att, str_list in straight_list.items() if any(all(die.count(x) >= s.count(x) for x in s) for s in str_list)
    ]
    return valid_attacks + straights



# Sort and return a tuple, so (1,2)->(1,2) and (2,1)->(1,2)
def sort_tuple(t:tuple):
    t = list(t)
    t.sort()
    return tuple(t)

# Helper function to return which dice are unchanged given a reroll
def unchanged(die:list, reroll:list):
    unch = []
    die_count = Counter(die)
    reroll_count = Counter(reroll)
    for k in reroll_count:
        die_count[k] -= reroll_count[k]
    for k in die_count:
        unch.extend([k] * die_count[k])
    return tuple(unch)

# Reroll k dice, return a list of tuples, rerolled dice, unchanged dice
def reroll_combs_k(die:list[int],k):
    die.sort()
    reroll = list(combinations(die,k))
    reroll = list(set([sort_tuple(t) for t in reroll]))
    unch = [unchanged(die, r) for r in reroll]
    return reroll, unch

# Calculate the possible reroll combinations
# Return a pair of lists of tuples, rerolled dice, unchanged dice
def reroll_combs(die:list[int]):
    reroll = []
    unch = []
    for i in range(6):
        r, u = reroll_combs_k(die, i)
        reroll.append(r)
        unch.append(u)
    return [reroll, unch]

#Brute force for now, given a list of die, roll the rest and return all the possible outcomes, including dups
def roll_dice(die:list):
    k = 5 - len(die)
    answer = []
    for p in product(range(1,7), repeat=k):
        die_p = die + list(p)
        die_p.sort()
        answer.append(die_p)
    return answer

#Given a list of dice roll permutations, calculate the odds of rolling different attacks,
#and chance of hitting something
def roll_odds(die_p:list):
    denom = len(die_p)
    probs = []
    attacks = defaultdict(int)
    hit = 0

    for d in die_p:
        attlist = valid_attacks(d)
        for a in attlist:
            attacks[a] += 1
        if attlist:
            hit += 1

    for att, cnt in attacks.items():
        probs.append([att, cnt/denom])

    probs.append(["Hit anything", hit / denom])

    return probs

#Given the current roll, calculate the moves that give the best odds of hitting each move
def find_best_move(die):

    best_odds = {}

    reroll_lol, unch_lol = reroll_combs(die)

    for reroll_list, unch_list in zip(reroll_lol, unch_lol):
        for reroll, unch in zip(reroll_list, unch_list):
            d_perms = roll_dice(list(unch))
            probs = roll_odds(d_perms)
            for att, prob in probs:
                if not best_odds.get(att, []) or prob > best_odds[att][0][0]:
                    best_odds[att] = [[prob, unch, reroll]]
                elif prob == 1:
                    best_odds[att].append([prob, unch, reroll])

    return best_odds

    

def best_move(die):
    
    best_odds = find_best_move(die)
    best_moves = []
    for attack, move_list in best_odds.items():
        best_moves.append([attack, move_list])

    best_moves.sort(key = lambda x: x[1][0], reverse = True)

    for attack, move_list in best_moves:
        for prob, unch, reroll in move_list:
            print(f'{attack} has {prob*100:.2f}% chance of success. Keep {unch} and reroll {reroll}')

best_move([1,1,4,4,6])


Hit anything has 93.52% chance of success. Keep (1, 1) and reroll (4, 4, 6)
Fireball1 has 87.50% chance of success. Keep (1, 1) and reroll (4, 4, 6)
Pyroblast has 75.00% chance of success. Keep (1, 1, 6) and reroll (4, 4)
Fireball2 has 50.00% chance of success. Keep (1, 1) and reroll (4, 4, 6)
Combustion has 33.33% chance of success. Keep (4, 6) and reroll (1, 1, 4)
Hot Streak has 21.30% chance of success. Keep (4,) and reroll (1, 1, 4, 6)
Burning Soul has 19.62% chance of success. Keep () and reroll (1, 1, 4, 4, 6)
Fireball3 has 12.50% chance of success. Keep (1, 1) and reroll (4, 4, 6)
Ignite has 3.70% chance of success. Keep (4,) and reroll (1, 1, 4, 6)
Meteorite has 1.62% chance of success. Keep (6,) and reroll (1, 1, 4, 4)
Ultimate has 0.08% chance of success. Keep (6,) and reroll (1, 1, 4, 4)


In [65]:
# Using input in Jupyter
user_input = input("Enter a number: ")
print(f"You entered: {user_input}")


You entered: hello


In [6]:
# Combination with repetition formula of n states, r repetitions
# C(n+r-1,r)

print(math.comb(6+5-1,5))
print(math.comb(4+5-1,5))

252
56


In [26]:
die = [1,2,3,4,5,6]
dtype = ['FLAME', 'BLAZE', 'FIERY_SOUL', 'METEOR']
print(list(combinations_with_replacement(die,5)))
print(len(list(combinations_with_replacement(die,5))))

print(list(combinations_with_replacement(dtype,5)))
print(len(list(combinations_with_replacement(dtype,5))))


[(1, 1, 1, 1, 1), (1, 1, 1, 1, 2), (1, 1, 1, 1, 3), (1, 1, 1, 1, 4), (1, 1, 1, 1, 5), (1, 1, 1, 1, 6), (1, 1, 1, 2, 2), (1, 1, 1, 2, 3), (1, 1, 1, 2, 4), (1, 1, 1, 2, 5), (1, 1, 1, 2, 6), (1, 1, 1, 3, 3), (1, 1, 1, 3, 4), (1, 1, 1, 3, 5), (1, 1, 1, 3, 6), (1, 1, 1, 4, 4), (1, 1, 1, 4, 5), (1, 1, 1, 4, 6), (1, 1, 1, 5, 5), (1, 1, 1, 5, 6), (1, 1, 1, 6, 6), (1, 1, 2, 2, 2), (1, 1, 2, 2, 3), (1, 1, 2, 2, 4), (1, 1, 2, 2, 5), (1, 1, 2, 2, 6), (1, 1, 2, 3, 3), (1, 1, 2, 3, 4), (1, 1, 2, 3, 5), (1, 1, 2, 3, 6), (1, 1, 2, 4, 4), (1, 1, 2, 4, 5), (1, 1, 2, 4, 6), (1, 1, 2, 5, 5), (1, 1, 2, 5, 6), (1, 1, 2, 6, 6), (1, 1, 3, 3, 3), (1, 1, 3, 3, 4), (1, 1, 3, 3, 5), (1, 1, 3, 3, 6), (1, 1, 3, 4, 4), (1, 1, 3, 4, 5), (1, 1, 3, 4, 6), (1, 1, 3, 5, 5), (1, 1, 3, 5, 6), (1, 1, 3, 6, 6), (1, 1, 4, 4, 4), (1, 1, 4, 4, 5), (1, 1, 4, 4, 6), (1, 1, 4, 5, 5), (1, 1, 4, 5, 6), (1, 1, 4, 6, 6), (1, 1, 5, 5, 5), (1, 1, 5, 5, 6), (1, 1, 5, 6, 6), (1, 1, 6, 6, 6), (1, 2, 2, 2, 2), (1, 2, 2, 2, 3), (1, 2, 2, 2, 